# ssHiCstuff — **Design tutorial (Bioinformatics paper companion)**

This notebook demonstrates the **annealing oligonucleotide design** workflow for ssHi-C,
with a **toy dataset** and fully documented steps. It mirrors the CLI command:
`sshicstuff design` and the internal wrapper logic described in the paper.

> Focus: *Design only.* A separate notebook will cover **Analysis** (filtering, profiles, stats, aggregation).

## Objectives

1. **Input preparation** — genome FASTA and target intervals.  
2. **Run design** — call the Rust backend `oligo4sshic` via the Python wrapper.  
3. **Genome edition** — build the `chr_artificial_ssDNA` FASTA and a concatenated `*_artificial.fa`.  
4. **Capture oligos** — derive capture sequences from annealing oligos (5' & 3' deletions).  
5. **Outputs recap** — CSV/FASTA artifacts and how they will be used downstream by the pipeline.

### Mini-glossary (design)

- **Annealing oligo**: short ssDNA sequence that **restores a restriction site** (e.g. DpnII `GATC`) in an **ssDNA** region.
- **Capture oligo**: derived from the annealing oligo (trimmed ends) used for targeted enrichment.
- **Secondary sites**: optional sequence changes to mutate other restriction sites (to suppress dsDNA signal).


In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ.pop("MPLBACKEND", None)

import sys
from sshicstuff import main as _entrypoint
from Bio import SeqIO
import pandas as pd
from os.path import join

## 1) Inputs — toy dataset

Provide the **full set of inputs** expected by `sshicstuff design`. This notebook uses a minimal, valid toy setup, but every parameter below is available exactly like in the CLI.

### Core inputs
- `--genome` (**required**): genome FASTA file (reference)
- `--forward-intervals`: comma-separated windows for forward oligos (e.g. `chr5:118710-133000`)
- `--reverse-intervals`: comma-separated windows for reverse oligos (e.g. `chr5:100000-118710`)
- `--site`: main restriction motif (default: `GATC` for DpnII)
- `--secondary-sites`: motifs to mutate to suppress dsDNA recutting (default: `CAATTG,AATATT,GANTC`)

### Oligo geometry & constraints
- `--size`: total oligo length (nt) (default: `75`)
- `--site-start`: 1-based index of the restriction site within the oligo (default: `65`)
- `--no-snp-zone`: protected region around the site; no SNPs there (default: `5`)
- `--complementary-size`: complementary overlap used for annealing (default: `7`)
- `--snp-number`: max synthetic SNPs per oligo (default: `5`)
- `--tries`: how many attempts to find valid candidates (default: `20`)
- `-v/--verbose`: verbose mode

### Outputs & locations
- `--outdir` (**required**): base directory for all outputs (CSV/FASTA)
- `--o4s-output-raw`: raw oligos (FASTA) from the **binary** `oligo4sshic`
- `--o4s-output-snp`: SNP oligos (FASTA) from the **binary** `oligo4sshic`
- `--annealing-csv`: processed annealing table (CSV) (wrapper)
- `--capture-csv`: processed capture oligos table (CSV) (wrapper)

### Genome edition (wrapper stage)
- `--n-5-prime-deletion`, `--n-3-prime-deletion`: trimming for **capture** oligos (default: `10`, `10`)


In [3]:
# === INPUTS ===
DATA_DIR = "../test_data/"
GENOME_NAME = "S288c_DSB"
FASTA = join(DATA_DIR, "inputs", "S288c_DSB.fa")

# === OUTPUT DIRECTORY & PREFIX ===
OUTDIR = join(DATA_DIR, "design-outputs")

# === OUTPUT FILENAMES ===
O4S_OUTPUT_RAW  = "output_o4s_raw.fa"
O4S_OUTPUT_SNP  = "output_o4s_snp.fa"
ANNEALING_CSV   = "annealing_oligo_positions.csv"
CAPTURE_CSV     = "capture_oligo_positions.csv"

# === DESIGN COMMAND ARGUMENTS ===
ARGS = [
    "-f", f"{FASTA}",
    "--forward-intervals", "chr5:118710-133000",
    "--reverse-intervals", "chr5:100000-118710",
    "--site", "GATC",
    "--secondary-sites", "CAATTG,AATATT",
    "--size", "80",
    "--site-start", "70",
    "--n-5-prime-deletion", "10",
    "--n-3-prime-deletion", "10",
    "--outdir", f"{OUTDIR}",
    "--o4s-output-raw", f"{O4S_OUTPUT_RAW}",
    "--o4s-output-snp", f"{O4S_OUTPUT_SNP}",
    "--annealing-csv", f"{ANNEALING_CSV}",
    "--capture-csv", f"{CAPTURE_CSV}",
]

In [4]:
# RUNNING DESIGN
_sys_argv = ["sshicstuff", "design"] + ARGS
sys.argv = _sys_argv
_entrypoint.main()

INFO :: [Design/Oligo4sshic] Running backend: oligo4sshic --fasta ../test_data/inputs/S288c_DSB.fa --forward-intervals chr5:118710-133000 --reverse-intervals chr5:100000-118710 --site GATC --secondary-sites CAATTG,AATATT --size 80 --site-start 70 --no-snp-zone 5 --complementary-size 7 --snp-number 5 --tries 20 --output-raw /home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs/output_o4s_raw.fa --output-snp /home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs/output_o4s_snp.fa


rerverse oligo: 44 / 90


INFO :: [Design/Annealing] Formatting annealing oligo output from FASTA to CSV ...
INFO :: [Design/EditGenome] Start. Enzyme=GATC | spacer N=150 | outdir=/home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs
INFO :: [Design/EditGenome] Loaded oligos: ss=12 | ds=0
INFO :: [Design/EditGenome] Building artificial chromosomes (ssDNA & dsDNA) ...
INFO :: [Design/EditGenome] Built ssDNA (snp) and dsDNA (no snp) artificial, lengths = 4792 bp
INFO :: [Design/EditGenome] Writing artificial chromosomes to: /home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs/chr_artificial_ssDNA.fa, /home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs/chr_artificial_dsDNA.fa
INFO :: [Design/EditGenome] Masking original oligo regions with N in S288c_DSB.fa ...
INFO :: [Design/EditGenome] Appending artificial chromosomes and writing /home/adminico/Documents/projects-src/sshicstuff/test_data/design-outputs/S288c_DSB_edited.fa
INFO :: [Design/EditGenome] 

## Reference Genome Edition and Artificial Chromosomes

### 1. Artificial chromosomes

Two artificial chromosomes are created to separate **ssDNA** from **dsDNA** signals at the mapping step:

- **`chr_artificial_dsDNA`** → concatenation of the *original (unmodified)* annealing oligo sequences  
  → represents the **double-stranded control** (no SNPs introduced)

- **`chr_artificial_ssDNA`** → concatenation of the *modified (SNP-containing)* annealing oligo sequences  
  → represents the **single-stranded restored** regions used during ssHi-C

Both sequences are written as separate FASTA entries and later appended to the reference genome  
(`*_edited.fa` → includes masked native chromosomes + both artificial ones).

---

### 2. Separation by `N` spacers

Between each probe, blocks of `N` bases (default: `n_artificial_spacer = 150`) are inserted on both sides of the enzyme site:

...NNNNN + [enzyme] + NNNNN + [probe sequence] + NNNNN + [enzyme] + NNNNN...


This has two key effects:

1. **Ensures one fragment per probe** → the number of restriction fragments equals the number of designed probes.  
2. **Prevents overlaps** between restriction enzyme motifs and probe sequences,  
   guaranteeing that digestion and mapping remain consistent.

---

### 3. Masking the original probe regions

In the *edited genome*, the **original genomic coordinates** of each annealing oligo are replaced with a stretch of `N`s:

ATCGATCGATCG → NNNNNNNNNNNNNN

This step "silences" the original locus, ensuring that reads no longer align to the native site.  
Instead, alignment will occur **only on the artificial chromosomes**, where the modified sequences are located.

---

### 4. SNP vs. non-SNP logic

- The **ssDNA artificial chromosome** (`chr_artificial_ssDNA`) uses the **modified** sequences with introduced SNPs.  
- The **dsDNA artificial chromosome** (`chr_artificial_dsDNA`) uses the **original** (unmodified) sequences.

SNPs act as **molecular barcodes** to discriminate between ssDNA- and dsDNA-derived contacts during read mapping.

---

### 5. Practical outcome

After running the `design` command:

- The output folder contains:
  - `chr_artificial_dsDNA.fa`  
  - `chr_artificial_ssDNA.fa`  
  - `<genome>_edited.fa` (original genome + both artificial chromosomes)

Mapping reads to this edited genome enables **competitive alignment**:
- Reads mapping to `chr_artificial_ssDNA` → ssDNA-specific contacts  
- Reads mapping to `chr_artificial_dsDNA` → dsDNA controls

This design ensures clean separation of molecular origins in downstream ssHi-C analysis.


In [5]:
## Printing ssDNA artificial chromosome
for record in SeqIO.parse(join(OUTDIR, "chr_artificial_ssDNA.fa"), "fasta"):
    print(f"ID: {record.id}")
    print(f"Description: {record.description}")
    print(f"Sequence: {record.seq}")

ID: chr_artificial_ssDNA
Description: chr_artificial_ssDNA (4792 bp)
Sequence: NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGATCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNCCGCACCCTTTCCAATAACAATTAGAATATGTTTATTGTAATCTTGCAAAATAAGCTGTACGACTTTTTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGATCNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGTAATCTTGTCTATATTGTCAGGGGTTAACATACCGGTAAACGTCAAAAATGGGGACAAAAGACTAACTTNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGATCNNNNNNNNNNNNNNNNNNN

## From Annealing Oligos to Capture Oligos

### 1. Purpose

After generating and integrating the **annealing oligonucleotides**, a second set of oligos — the **capture oligonucleotides** — is derived.  
These are shorter versions used in the hybridization step of the ssHi-C capture protocol.

---

### 2. Principle

Each capture oligo is built from its corresponding **modified (SNP)** annealing sequence by trimming a few bases at both ends:

Capture = Annealing[ +n₅′ : −n₃′ ]


where:

- `n₅′` → number of bases removed from the **5′ end** (default = 10)  
- `n₃′` → number of bases removed from the **3′ end** (default = 10)

This trimming ensures that:
- The **capture oligo** hybridizes specifically to the restored region (SNP-containing),
- But **does not include** the primary restriction site sequence itself,
- Avoiding interference with enzyme recognition or digestion.

---

### 3. Workflow summary

1. **Input**: `annealing_oligo_positions.csv`  
   (contains both `sequence_original` and `sequence_modified` columns)
2. **Processing**: `annealing_to_capture()` trims each modified sequence  
   according to `--n-5-prime-deletion` and `--n-3-prime-deletion`.
3. **Output**: `capture_oligo_positions.csv`  
   (ready for synthesis or capture probe ordering)

---

### 4. Conceptual difference

| Oligo type | Sequence source | Contains SNPs | Includes restriction site | Purpose |
|-------------|----------------|----------------|----------------------------|----------|
| **Annealing** | Designed to restore the cut site | ✅ Yes | ✅ Yes | Restores enzyme site for ssHi-C ligation |
| **Capture** | Derived from modified annealing sequence | ✅ Yes | ❌ No | Enriches reads from the restored ssDNA region |


In [7]:
annealing_oligos_path = join(OUTDIR, "annealing_oligo_positions.csv")
capture_oligos_path = join(OUTDIR, "capture_oligo_positions.csv")
df_annealing = pd.read_csv(annealing_oligos_path, sep=",", header=0)
df_capture = pd.read_csv(capture_oligos_path, sep=",", header=0)
print(f"Found {len(df_capture)} oligos")

Found 12 oligos


In [12]:
df_annealing.head()

,chr,start,end,length,chr_ori,start_ori,end_ori,orientation,type,name,sequence_original,sequence_modified
0,chr_artificial_ssDNA,150,524,374,chr5,121385.0,121464.0,w,ss,Probe_chr5_w_121385_121464,CCGCACCCTTTCCAATAACAATCAGAATATTTTTATTTTTATGTTG...,CCGCACCCTTTCCAATAACAATTAGAATATGTTTATTGTAATCTTG...
1,chr_artificial_ssDNA,524,898,374,chr5,121624.0,121703.0,w,ss,Probe_chr5_w_121624_121703,GTAATCATGTCAATATTGTCAGGGGTTAACTTTCCGGTAAACTTCA...,GTAATCTTGTCTATATTGTCAGGGGTTAACATACCGGTAAACGTCA...
2,chr_artificial_ssDNA,898,1272,374,chr5,126743.0,126822.0,w,ss,Probe_chr5_w_126743_126822,TATCGTCATATCTGTGCTTTCTGTTATCGTATTGGAAATATTTCCA...,TATCGTCATATCTGTGCTTTCTGTTATAGTCTTGGAAATTTTTCCA...
3,chr_artificial_ssDNA,1272,1646,374,chr5,130786.0,130865.0,w,ss,Probe_chr5_w_130786_130865,TACTGAAAAATACGTCCGTCAGGTCTCTAGAGAGGTACTGGAACCC...,TACTGAAAAATACGTCCGTCAGGTCTCTCGAGAGGTACTGGAACCC...
4,chr_artificial_ssDNA,1646,2020,374,chr5,132710.0,132789.0,w,ss,Probe_chr5_w_132710_132789,CGTTTTTAGAATATATTGTAATAAAACACAATTGATAATACAGTTC...,CGTTTTTAGCATATATTGTATTAAAACAGGATTGATAATACAGTTC...


In [11]:
df_capture.head()

,chr,start,end,chr_ori,start_ori,end_ori,orientation,type,name,sequence
0,chr_artificial_ssDNA,150,524,chr5,121385.0,121464.0,w,ss,Probe_chr5_w_121385_121464,TCCAATAACAATTAGAATATGTTTATTGTAATCTTGCAAAATAAGC...
1,chr_artificial_ssDNA,524,898,chr5,121624.0,121703.0,w,ss,Probe_chr5_w_121624_121703,CTATATTGTCAGGGGTTAACATACCGGTAAACGTCAAAAATGGGGA...
2,chr_artificial_ssDNA,898,1272,chr5,126743.0,126822.0,w,ss,Probe_chr5_w_126743_126822,TCTGTGCTTTCTGTTATAGTCTTGGAAATTTTTCCAGGTCGGGTCG...
3,chr_artificial_ssDNA,1272,1646,chr5,130786.0,130865.0,w,ss,Probe_chr5_w_130786_130865,TACGTCCGTCAGGTCTCTCGAGAGGTACTGGAACCCATCTTACTAG...
4,chr_artificial_ssDNA,1646,2020,chr5,132710.0,132789.0,w,ss,Probe_chr5_w_132710_132789,ATATATTGTATTAAAACAGGATTGATAATACAGTTCTCTCTTCGTC...
